# Manipulando dados com Pandas

Saber manipular dados e realizar algumas transformações neles é essencial. Nesse tutorial, iremos passar por funções importantes da biblioteca Pandas e utilizar o dataset de "Inauguração de Estações de Metrô e da CPTM" para exemplificá-las.

Para esse tutorial, é necessário conhecer o básico de Python (funções, bibliotecas, etc).

Tópicos
- como carregar dados
- valores nulos
- filtros
- ver máximo, média, mediana
- contar valores, ordenação

Bibliotecas que iremos utilizar:

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import os

# Introdução

Para carregar os dados com a biblioteca Pandas, iremos utilizar o método ```pd.read_csv()```.
Basta passarmos para a função o local onde se encontram os dados, que no nosso caso, são do tipo ```.csv``` (outros parâmetros podem ser encontrados na [documentação](pandas.pydata.org/)).
O Pandas carrega nossos dados em tipo chamado ```DataFrame```.

Para verificarmos o tamanho do dataset utilizamos o ```.shape``` em nosso DataFrame, ou seja, ```data.shape```.

Também, queremos ver um trecho dos nossos dados, e podemos ver os 5 primeiros itens com ```data.head()```.

In [ ]:
data = pd.read_csv('/kaggle/input/inauguracao-de-estacoes-do-metrosp-e-da-cptm/inauguracoes.csv')
print(f'Dataset shape -> {data.shape}')
data.head()

Para acessar o valor de uma linha, não fazemos como nas listas, de selecionar um elemento pelo index. Veja saída, caso tentemos fazer isso:

In [ ]:
data[0]

O erro nos diz essa chave não existe, porque na verdade, a busca procura pelas colunas do dataset.

Para acessar uma linha pelo valor de sua posição, podemos usar a função ```data.iloc[idx]```:

In [ ]:
data.iloc[0]

Para uma descrição das colunas numéricas, com algumas medidas de posição, máximo e mínimo, podemos executar ```data.describe()```:

In [ ]:
data.describe()

Seria possível obter esses valores por meio das funções de máximo, mínimo, desvio padrão e dos quantis:

In [ ]:
print(f'Média: {data.Linha.mean()}')
print(f'Desvio: {data.Linha.std()}')
print(f'Max: {data.Linha.max()}')
print(f'Min: {data.Linha.min()}')
print(f'Quantis (25%, 50%, 75%, 100%): {data.Linha.quantile(.25)}, {data.Linha.quantile(.5)}, {data.Linha.quantile(.75)}, {data.Linha.quantile(1)}')

Podemos também querer informações sobre todas as colunas, com informações sobre valores nulos, tipo de dados da coluna. A função ```data.info()``` permite isso:

In [ ]:
data.info()

# Valores nulos

Um ponto bem importante para competições é verificarmos valores nulos. Tanto para que medidas façam mais sentido (e.g. a média) quanto para nossos modelos, valores nulos podem impactar nossa avaliação, portanto, é necessário tratá-los de alguma forma. Existem diferentes técnicas para inputar dados, que não iremos abordar aqui, mas valem a [referência](https://scikit-learn.org/stable/modules/impute.html).

Um passo inicial, é verificar a existência de valores nulos. O método ```data.info()``` permite que visualizemos a existência deles por colunas, o que pode ser chato de ver quando o nosso número de colunas é grande. Uma forma mais prática é utilizar o método do Pandas ```data.isna()```.

In [ ]:
data.isna()

Vemos acima que ele verifica o valor de uma célula, retornando verdadeiro ou falso caso ela seja nula (ou não). Ainda não é o que queremos.

O que falta é chamar o método ```sum()```, para que os valores sejam somados (verdadeiro = 1 e falso = 0), e assim checarmos por coluna a existência de nulos:

In [ ]:
data.isna().sum()

Uma outra alternativa, para checar a existência de nulos entre todas as colunas, é chamar o método ```sum()``` mais uma vez:

In [ ]:
data.isna().sum().sum()

Por fim, verificamos que não há nenhum dado faltante em nosso dataset.

# Unicidade e filtros

Uma pergunta interessante de se fazer, considerando nossos dados, seria saber quais são as estações de Metrô ou CPTM que foram inauguradas.
Temos a coluna ```Nome``` que diz o nome de uma estação, mas não sabemos se esse nome pode estar em mais de uma linha.

Para isso, podemos utilizar o método ```data.Nome.unique()```, que nos diz os valores únicos naquela coluna.
Também, podemos querer saber apenas o número de valores únicos nessa coluna, o que pode ser feito com o método ```data.Nome.nunique()```:

In [ ]:
print(f'Number of unique stations: {data.Nome.nunique()}')
data.Nome.unique()

Podemos perceber que nosso dataset possui duplicatas, já que são 169 estações mas 184 linhas.

Quais são as linhas duplicadas?
O método ```data.Nome.duplicated()``` pode nos ajudar com essa pergunta. Ele retorna verdadeiro ou falso.

Como queremos observar todas as informações dessas estações duplicadas, queremos as linhas que são ```True``` no retorno desse método.
Aqui entram os filtros!

# Filtros

Podemos filtrar nosso dataset com o retorno booleano (verdadeiro ou falso) do método ```data.Nome.duplicated()``` da seguinte maneira:

In [ ]:
print(f'O que a função duplicated retorna: \n{data.Nome.duplicated()}\n____')

print('Estações duplicadas:')
data[data.Nome.duplicated()]

Para investigar mais a fundo as informações de uma estação que está duplicada, podemos selecionar algumas delas, e filtrá-las pelo nome.

No Pandas, podemos encadear filtros (com ```or``` e ```and```), mas a sintaxe muda:
1. ```or``` -> |
2. ```and``` -> &
3. ```not``` -> ~

Por exemplo, iremos filtrar:
1. Estações com nome Luz ou Santa Cruz
2. Estações com nome Luz e construídas pelo Metrô
3. Estações com nome Luz e não construídas pelo Metrô

In [ ]:
data[(data.Nome == 'Luz') | (data.Nome == 'Santa Cruz')]

In [ ]:
data[(data.Nome == 'Luz') & (data.Construção == 'Metrô')]

In [ ]:
data[(data.Nome == 'Luz') & ~(data.Construção == 'Metrô')]

Também pode ser o caso em que desejamos retirar os valores duplicados, o que pode ser feito com a função ```data.drop_duplicates()```.

A função remove o item duplicado, levando em consideração a coluna de comparação. Ela retorna um novo ```DataFrame``` caso o parâmetro ```inplace``` não seja utilizado (```inplace=True```).

Um parâmetro importante dessa função é o ```keep```. Como padrão, o Pandas mantém a primeira ocorrência daquele valor, mas podemos alterá-lo para manter o último.
Para nosso estudo, isso acaba não fazendo tanta diferença.

In [ ]:
data_without_duplicates = data.drop_duplicates(subset=['Nome'], keep='last')
data_without_duplicates.Nome.duplicated().sum()

Os filtros também podem ser aplicados as demais colunas.

Temos abaixo o exemplo com as idades das estações. Podemos querer aquelas com mais de 40 anos (mais antigas), ou com menos de 4 anos (mais recentes).

In [ ]:
data[data.Idade > 40]

In [ ]:
data[data.Idade < 4]

# Conversões de tempo (datetime)

Saber lidar com dados temporais é importante. O primeiro passo é convertê-los para o tipo certo, facilitando a sua manipulação, construção de novas
features e até mesmo que realizemos filtros baseados em tempo.

No nosso dataset, temos a coluna de Inauguração, que representa a data de inauguração de uma estação. Podemos observar que o tipo dela é ```object```:

In [ ]:
data.Inauguração.head()

Iremos convertê-la para ```datetime``` com a função do Pandas ```pd.to_datetime()```:

In [ ]:
data['Inauguração'] = pd.to_datetime(data['Inauguração'])
data.Inauguração.head()

Dessa forma, podemos derivar novas features com facilidade:

In [ ]:
data['Ano'] = pd.DatetimeIndex(data.Inauguração).year
data['Mês'] = pd.DatetimeIndex(data.Inauguração).month
data['Dia'] = pd.DatetimeIndex(data.Inauguração).day
data.head()

# Ordenação de valores

Outro ponto que é interessante é conseguirmos ordenar nossos dados.

Isso pode ser feito tanto para ```Series``` quanto ```DataFrame```. Podemos ver alguns exemplos abaixo,
que levam em consideração a data de inauguração da estação:

In [ ]:
data.Inauguração.sort_values()

In [ ]:
data.sort_values(by=['Inauguração'])

Além da ordenação levando em consideração o tempo, podemos ordenar por qualquer outra coluna, como por exemplo, pelo nome:

In [ ]:
data.sort_values(by=['Nome'])

Lembre-se: a função ```sort_values``` também retorna um novo ```DataFrame/Series```, e você deve atribuí-lo caso deseje usar esse novo df. Também é possível fazer isso ```inplace```.

Essa função também pode considerar diferentes eixos na hora de ordenar (linhas ou colunas).

# Criando novas colunas (features)

O processo de criar novas features (características) também é essencial. Muitas vezes, derivar uma nova feature pode melhorar (e muito) a performance do seu modelo.
Ela eleva a "riqueza" das informações a serem utilizadas.

Existem formas mais programáticas de gerar features, como [features polinomiais](https://scikit-learn.org/stable/modules/preprocessing.html#generating-polynomial-features), mas iremos fazer isso
de uma forma um pouco mais manual, apenas para dar uma ideia/sugestão.

Dentro do nosso contexto, poderiamos estar trabalhando em um problema de decidir a qualidade ou eficiência de uma estação de metrô.
Pensando nesse problema (imaginário), iriamos nos perguntar:

1. Será que a estação ser da CPTM faz diferença?
2. Ser antiga ou nova muda algo?

Outros pontos, que talvez fossem meio inusitados de se perguntar, mas eventualmente poderiam evidenciar algo não facilmente percebido nos dados, seria:

1. O nome da estação contém z?
2. A idade da estação é par?
3. Ela foi inaugurada no mês do Natal?
4. Ela foi inaugurada no segundo semestre?
5. Ela foi inaugurada nas férias (férias sendo janeiro e julho)?

Cada uma dessas perguntas poderia gerar uma nova feature, como exemplificado abaixo:

In [ ]:
data['isCPTM'] = (data.Construção == 'CPTM').astype('int')
data.head()

In [ ]:
data['isOld'] = (data.Idade > 40).astype('int')
data['hasZ'] = data.Nome.str.contains('z').astype('int')
data['ageIsEven'] = (data.Idade % 2 == 0).astype('int')
data['yearIsOdd'] = (data.Ano % 2 != 0).astype('int')
data['isChristmasMonth'] = (data.Mês == 12).astype('int')
data['isSecondSemester'] = (data.Mês >= 6).astype('int')
data['isVacation'] = ((data.Mês == 7) | (data.Mês == 1)).astype('int')

data.head()

# Contagem de valores

Por fim, iremos verificar como contar a ocorrência de valores em nosso dataset.

A função ```value_counts()``` permite que façamos isso. Basta passar uma coluna:

In [ ]:
data.Ano.value_counts()

Como resultado, obtemos o número de ocorrências de cada um dos valores, o que pode ser particularmente útil para visualizações, verificarmos se os dados estão desbalanceados (mais de uma classe, menos das outras) dentro do contexto de classificação, e mais.

Como exemplo, iremos visualizar esses dados (não se preocupe em como fazer a visualização):

In [ ]:
xy = data.Ano.value_counts()

plt.figure(figsize=(50,10))
plt.bar(xy.index.astype('str'), height=xy.values);

# FIM

Nesse notebook, passamos por alguns dos pontos mais importantes quando falamos de manipulação de dados.
Vimos como descrever uma ou mais colunas, checar se ela contém valores nulos, como acessar os valores de um dataset.
Também, como criar novas colunas derivadas de informações que já temos (feature engineering), lidar com dados de tempo, ordenar e filtrar valores.

Se você gostou do notebook, dê um up :)

Sugestões de melhoria, críticas e qualquer mensagem que quiser passar, deixe nos comentários!


Esse notebook foi criado para uma apresentação no GT Estudos do [BeeData](https://www.facebook.com/BeeDataUSP/). Fique à vontade para entrar em contato!